#### Importing the libraries

In [1]:
# link :https://archive.ics.uci.edu/ml/datasets/ecoli
import numpy as np 
import csv
from keras.layers import Input, Dense, Reshape, Flatten
from keras.layers import BatchNormalization, Activation
from keras.layers.advanced_activations import LeakyReLU
from keras.layers.convolutional import UpSampling2D, Conv2D
from keras.models import Model
from keras import backend as K
from keras.optimizers import Adam

Using TensorFlow backend.


#### Importing Data

In [2]:
raw_data = open('ecoli.csv', 'rt')
reader = csv.reader(raw_data, delimiter=',', quoting=csv.QUOTE_NONE)
data = list(reader)
data = np.array(data)
data = data[:,2: -2].astype(float)

In [3]:
print(data)
print(np.shape(data))

[[0.29 0.48 0.5  0.56 0.24]
 [0.4  0.48 0.5  0.54 0.35]
 [0.4  0.48 0.5  0.49 0.37]
 ...
 [0.6  0.48 0.5  0.44 0.39]
 [0.61 0.48 0.5  0.42 0.42]
 [0.74 0.48 0.5  0.31 0.53]]
(336, 5)


#### Data Processing
- 
    Sequences are made into a square image of closest binary number to the the ceiling of the root of the length


In [4]:
rows, columns = np.shape(data)
sqrt_col = int(np.ceil(np.sqrt(columns)))
processed_data = np.zeros([rows, 3, 3])
data_ = np.zeros([rows, 3*3])

for i in range(rows):
#     data[i] = (data[i] - np.mean(data[i]))
    data_[i] = np.append(data[i], np.zeros(3*3 - columns))

    
for i in range(rows):
    for j in range(sqrt_col):
            processed_data[i,j,:] = data_[i, j * 3 : (j+1)*3]

In [5]:
processed_data[0]

array([[0.29, 0.48, 0.5 ],
       [0.56, 0.24, 0.  ],
       [0.  , 0.  , 0.  ]])

#### Generator

In [6]:
def gen_model(input_shape):
    
    Input_1= Input(shape = input_shape)
    
    model = Conv2D(3, (3,3), activation='relu', input_shape=input_shape, padding ='same')(Input_1)
    model = BatchNormalization()(model)
    
    model = Conv2D(4, (4,4), activation='relu', input_shape=input_shape, padding ='same')(model)
    model = BatchNormalization()(model)
    
    model = Conv2D(4, (4,4), activation='relu', input_shape=input_shape, padding ='same')(model)
    model = BatchNormalization()(model)
    
    model = Conv2D(1, (4,4), activation='tanh', input_shape=input_shape, padding ='same')(model)
    output = BatchNormalization()(model)
    
    model = Model(inputs=Input_1, outputs = output, name = 'generator')
    
    return model

In [7]:
gen = gen_model((3,3,1))

In [8]:
gen.summary()

Model: "generator"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 3, 3, 1)           0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 3, 3, 3)           30        
_________________________________________________________________
batch_normalization_1 (Batch (None, 3, 3, 3)           12        
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 3, 3, 4)           196       
_________________________________________________________________
batch_normalization_2 (Batch (None, 3, 3, 4)           16        
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 3, 3, 4)           260       
_________________________________________________________________
batch_normalization_3 (Batch (None, 3, 3, 4)           16

#### Discriminator

In [9]:
def discrim_model(input_shape):
    
    Input_1= Input(shape = input_shape)
    
    model = Conv2D(3, (4,4), activation='relu', input_shape=input_shape, padding ='same')(Input_1)
    model = BatchNormalization()(model)

    model = Conv2D(3, (2,2), activation='relu', input_shape=input_shape, padding ='same')(model)
    model = BatchNormalization()(model)
    
    model = Conv2D(3, (2,2), activation='relu', input_shape=input_shape, padding ='same')(model)
    model = BatchNormalization()(model)

    model = Flatten()(model)
    
    output = Dense(1, activation = 'sigmoid')(model)
    
    model = Model(inputs=Input_1, outputs = output, name = 'discriminator')
    
    return model


In [10]:
discrim = discrim_model((3,3,1))

In [11]:
discrim.summary()

Model: "discriminator"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         (None, 3, 3, 1)           0         
_________________________________________________________________
conv2d_5 (Conv2D)            (None, 3, 3, 3)           51        
_________________________________________________________________
batch_normalization_5 (Batch (None, 3, 3, 3)           12        
_________________________________________________________________
conv2d_6 (Conv2D)            (None, 3, 3, 3)           39        
_________________________________________________________________
batch_normalization_6 (Batch (None, 3, 3, 3)           12        
_________________________________________________________________
conv2d_7 (Conv2D)            (None, 3, 3, 3)           39        
_________________________________________________________________
batch_normalization_7 (Batch (None, 3, 3, 3)         

#### Creating the WGAN with Gradient Penalties

In [12]:
def build_WGANgp(generator, discriminator, z_shape, img_shape, penaltyLambda):
    #### model
    # generator image(fake image)
    z = Input(shape=z_shape)
    f_img = generator(z)
    f_out = discriminator(f_img)
    # real image
    r_img = Input(shape=img_shape)
    r_out = discriminator(r_img)
    # average image
    epsilon = K.placeholder(shape=(None,1,1,1))
    a_img = Input(shape=(img_shape),
                  tensor = epsilon * r_img + (1-epsilon) * f_img)
    a_out = discriminator(a_img)

    #### loss
    # original critic(discriminator) loss
    r_loss = K.mean(r_out)
    f_loss = K.mean(f_out)
    # gradient penalty  <this is point of WGAN-gp>
    grad_mixed = K.gradients(a_out, [a_img])[0]
    norm_grad_mixed = K.sqrt(K.sum(K.square(grad_mixed), axis=[1,2,3]))
    grad_penalty = K.mean(K.square(norm_grad_mixed -1))
    penalty = penaltyLambda * grad_penalty
    # d loss
    d_loss = f_loss - r_loss + penalty
    
    #### discriminator update function
    d_updates = Adam(lr=1e-4, beta_1=0.5, beta_2=0.9). \
                get_updates(discriminator.trainable_weights,[],d_loss)
    d_train = K.function([r_img, z, epsilon],
                         [r_loss, f_loss, penalty, d_loss],
                         d_updates)
    
    #### generator update function
    g_loss = -1. * f_loss
    g_updates = Adam(lr=1e-4, beta_1=0.5, beta_2=0.9). \
                get_updates(generator.trainable_weights,[],g_loss)
    g_train = K.function([z], [g_loss], g_updates)

    return g_train, d_train


#### Creating the entire network

In [13]:
gen_shape = (3,3,1)
discrim_shape = (3,3,1)
lamda = 0.05 # Hyperparameter
# generator Model
generator = gen_model(gen_shape)
# discriminator Model
discriminator = discrim_model(discrim_shape)
# WGAN-gp Training Model
G_train, D_train = build_WGANgp(generator, discriminator, gen_shape, discrim_shape, lamda)

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where




#### Training

In [14]:
# Training Parameters
g_loss_list = []
r_loss_list = []
f_loss_list = []
f_r_loss_list = []
penalty_list = []
d_loss_list = []
batch_size = 336
iterations = 3000

# Training Data
X_train = np.expand_dims(processed_data, axis = 3)

# Labels
true_label = np.ones((batch_size, 1))
fake_label = np.zeros((batch_size, 1))

# Training
for epoch in range(iterations):
    
    # Training the Discriminator
    # Generator in
    z = np.random.normal(0.5, 0.25, (batch_size, X_train.shape[1], X_train.shape[2], 1))
    # Generator out Images
    f_imgs = generator.predict(z)
    # Real Images
    idx = np.random.randint(0, X_train.shape[0], batch_size)
    r_imgs = X_train[idx]
    # train the discriminator
    epsilon = np.random.uniform(size = (batch_size, 1,1,1))
    r_loss, f_loss, penalty, d_loss = D_train([r_imgs, z, epsilon])
    
    # Training the Discriminator
    # Generator in
    z = np.random.normal(0.5, 0.25, (batch_size, X_train.shape[1], X_train.shape[2], 1))
    # train the generator
    g_loss = G_train([z])
      
    #### Record of learning progress
    # loss
    r_loss_list.append(r_loss)
    f_loss_list.append(f_loss)
    f_r_loss_list.append(f_loss - r_loss)
    penalty_list.append(penalty)
    d_loss_list.append(d_loss)
    # generated image sumple
    print(f'iteration:{epoch} | d_loss:{d_loss:.3f} | g_loss:{sum(g_loss)/len(g_loss):.3f}')

iteration:0 | d_loss:0.050 | g_loss:-0.499
iteration:1 | d_loss:0.050 | g_loss:-0.499
iteration:2 | d_loss:0.050 | g_loss:-0.499
iteration:3 | d_loss:0.050 | g_loss:-0.499
iteration:4 | d_loss:0.050 | g_loss:-0.499
iteration:5 | d_loss:0.050 | g_loss:-0.499
iteration:6 | d_loss:0.049 | g_loss:-0.500
iteration:7 | d_loss:0.050 | g_loss:-0.500
iteration:8 | d_loss:0.050 | g_loss:-0.500
iteration:9 | d_loss:0.050 | g_loss:-0.500
iteration:10 | d_loss:0.050 | g_loss:-0.500
iteration:11 | d_loss:0.049 | g_loss:-0.500
iteration:12 | d_loss:0.050 | g_loss:-0.500
iteration:13 | d_loss:0.050 | g_loss:-0.500
iteration:14 | d_loss:0.050 | g_loss:-0.500
iteration:15 | d_loss:0.049 | g_loss:-0.500
iteration:16 | d_loss:0.050 | g_loss:-0.500
iteration:17 | d_loss:0.050 | g_loss:-0.500
iteration:18 | d_loss:0.049 | g_loss:-0.500
iteration:19 | d_loss:0.050 | g_loss:-0.500
iteration:20 | d_loss:0.050 | g_loss:-0.500
iteration:21 | d_loss:0.049 | g_loss:-0.500
iteration:22 | d_loss:0.049 | g_loss:-0.50

iteration:191 | d_loss:0.040 | g_loss:-0.497
iteration:192 | d_loss:0.040 | g_loss:-0.497
iteration:193 | d_loss:0.040 | g_loss:-0.497
iteration:194 | d_loss:0.040 | g_loss:-0.497
iteration:195 | d_loss:0.040 | g_loss:-0.496
iteration:196 | d_loss:0.040 | g_loss:-0.496
iteration:197 | d_loss:0.039 | g_loss:-0.496
iteration:198 | d_loss:0.040 | g_loss:-0.496
iteration:199 | d_loss:0.040 | g_loss:-0.496
iteration:200 | d_loss:0.040 | g_loss:-0.496
iteration:201 | d_loss:0.040 | g_loss:-0.496
iteration:202 | d_loss:0.039 | g_loss:-0.496
iteration:203 | d_loss:0.040 | g_loss:-0.496
iteration:204 | d_loss:0.040 | g_loss:-0.496
iteration:205 | d_loss:0.039 | g_loss:-0.496
iteration:206 | d_loss:0.039 | g_loss:-0.496
iteration:207 | d_loss:0.039 | g_loss:-0.496
iteration:208 | d_loss:0.039 | g_loss:-0.496
iteration:209 | d_loss:0.039 | g_loss:-0.496
iteration:210 | d_loss:0.040 | g_loss:-0.496
iteration:211 | d_loss:0.039 | g_loss:-0.496
iteration:212 | d_loss:0.039 | g_loss:-0.496
iteration:

iteration:375 | d_loss:0.031 | g_loss:-0.492
iteration:376 | d_loss:0.031 | g_loss:-0.492
iteration:377 | d_loss:0.031 | g_loss:-0.492
iteration:378 | d_loss:0.031 | g_loss:-0.492
iteration:379 | d_loss:0.031 | g_loss:-0.492
iteration:380 | d_loss:0.031 | g_loss:-0.492
iteration:381 | d_loss:0.031 | g_loss:-0.492
iteration:382 | d_loss:0.030 | g_loss:-0.492
iteration:383 | d_loss:0.031 | g_loss:-0.492
iteration:384 | d_loss:0.031 | g_loss:-0.492
iteration:385 | d_loss:0.031 | g_loss:-0.492
iteration:386 | d_loss:0.031 | g_loss:-0.492
iteration:387 | d_loss:0.031 | g_loss:-0.492
iteration:388 | d_loss:0.031 | g_loss:-0.492
iteration:389 | d_loss:0.030 | g_loss:-0.492
iteration:390 | d_loss:0.030 | g_loss:-0.492
iteration:391 | d_loss:0.031 | g_loss:-0.492
iteration:392 | d_loss:0.030 | g_loss:-0.492
iteration:393 | d_loss:0.030 | g_loss:-0.492
iteration:394 | d_loss:0.030 | g_loss:-0.492
iteration:395 | d_loss:0.030 | g_loss:-0.492
iteration:396 | d_loss:0.030 | g_loss:-0.492
iteration:

iteration:559 | d_loss:0.027 | g_loss:-0.492
iteration:560 | d_loss:0.028 | g_loss:-0.492
iteration:561 | d_loss:0.027 | g_loss:-0.492
iteration:562 | d_loss:0.028 | g_loss:-0.492
iteration:563 | d_loss:0.027 | g_loss:-0.492
iteration:564 | d_loss:0.027 | g_loss:-0.492
iteration:565 | d_loss:0.027 | g_loss:-0.492
iteration:566 | d_loss:0.027 | g_loss:-0.492
iteration:567 | d_loss:0.027 | g_loss:-0.492
iteration:568 | d_loss:0.027 | g_loss:-0.492
iteration:569 | d_loss:0.027 | g_loss:-0.492
iteration:570 | d_loss:0.028 | g_loss:-0.492
iteration:571 | d_loss:0.027 | g_loss:-0.492
iteration:572 | d_loss:0.027 | g_loss:-0.492
iteration:573 | d_loss:0.027 | g_loss:-0.492
iteration:574 | d_loss:0.028 | g_loss:-0.492
iteration:575 | d_loss:0.028 | g_loss:-0.492
iteration:576 | d_loss:0.028 | g_loss:-0.492
iteration:577 | d_loss:0.027 | g_loss:-0.492
iteration:578 | d_loss:0.028 | g_loss:-0.492
iteration:579 | d_loss:0.027 | g_loss:-0.492
iteration:580 | d_loss:0.028 | g_loss:-0.492
iteration:

iteration:742 | d_loss:0.033 | g_loss:-0.493
iteration:743 | d_loss:0.034 | g_loss:-0.493
iteration:744 | d_loss:0.033 | g_loss:-0.494
iteration:745 | d_loss:0.033 | g_loss:-0.493
iteration:746 | d_loss:0.033 | g_loss:-0.494
iteration:747 | d_loss:0.033 | g_loss:-0.493
iteration:748 | d_loss:0.033 | g_loss:-0.494
iteration:749 | d_loss:0.033 | g_loss:-0.494
iteration:750 | d_loss:0.033 | g_loss:-0.493
iteration:751 | d_loss:0.034 | g_loss:-0.494
iteration:752 | d_loss:0.033 | g_loss:-0.494
iteration:753 | d_loss:0.033 | g_loss:-0.494
iteration:754 | d_loss:0.034 | g_loss:-0.494
iteration:755 | d_loss:0.033 | g_loss:-0.494
iteration:756 | d_loss:0.034 | g_loss:-0.494
iteration:757 | d_loss:0.034 | g_loss:-0.494
iteration:758 | d_loss:0.034 | g_loss:-0.494
iteration:759 | d_loss:0.033 | g_loss:-0.494
iteration:760 | d_loss:0.034 | g_loss:-0.494
iteration:761 | d_loss:0.034 | g_loss:-0.494
iteration:762 | d_loss:0.034 | g_loss:-0.494
iteration:763 | d_loss:0.034 | g_loss:-0.494
iteration:

...

iteration:2202 | d_loss:0.057 | g_loss:-0.464
iteration:2203 | d_loss:0.056 | g_loss:-0.465
iteration:2204 | d_loss:0.056 | g_loss:-0.465
iteration:2205 | d_loss:0.058 | g_loss:-0.465
iteration:2206 | d_loss:0.057 | g_loss:-0.466
iteration:2207 | d_loss:0.058 | g_loss:-0.466
iteration:2208 | d_loss:0.057 | g_loss:-0.467
iteration:2209 | d_loss:0.057 | g_loss:-0.467
iteration:2210 | d_loss:0.058 | g_loss:-0.467
iteration:2211 | d_loss:0.058 | g_loss:-0.468
iteration:2212 | d_loss:0.057 | g_loss:-0.468
iteration:2213 | d_loss:0.058 | g_loss:-0.468
iteration:2214 | d_loss:0.058 | g_loss:-0.468
iteration:2215 | d_loss:0.058 | g_loss:-0.469
iteration:2216 | d_loss:0.058 | g_loss:-0.469
iteration:2217 | d_loss:0.057 | g_loss:-0.470
iteration:2218 | d_loss:0.058 | g_loss:-0.470
iteration:2219 | d_loss:0.059 | g_loss:-0.470
iteration:2220 | d_loss:0.058 | g_loss:-0.471
iteration:2221 | d_loss:0.059 | g_loss:-0.471
iteration:2222 | d_loss:0.058 | g_loss:-0.472
iteration:2223 | d_loss:0.059 | g_

iteration:2384 | d_loss:0.068 | g_loss:-0.508
iteration:2385 | d_loss:0.067 | g_loss:-0.508
iteration:2386 | d_loss:0.068 | g_loss:-0.508
iteration:2387 | d_loss:0.067 | g_loss:-0.508
iteration:2388 | d_loss:0.068 | g_loss:-0.508
iteration:2389 | d_loss:0.068 | g_loss:-0.509
iteration:2390 | d_loss:0.067 | g_loss:-0.508
iteration:2391 | d_loss:0.067 | g_loss:-0.508
iteration:2392 | d_loss:0.067 | g_loss:-0.508
iteration:2393 | d_loss:0.068 | g_loss:-0.509
iteration:2394 | d_loss:0.067 | g_loss:-0.509
iteration:2395 | d_loss:0.067 | g_loss:-0.509
iteration:2396 | d_loss:0.067 | g_loss:-0.509
iteration:2397 | d_loss:0.068 | g_loss:-0.509
iteration:2398 | d_loss:0.067 | g_loss:-0.509
iteration:2399 | d_loss:0.067 | g_loss:-0.509
iteration:2400 | d_loss:0.068 | g_loss:-0.509
iteration:2401 | d_loss:0.067 | g_loss:-0.509
iteration:2402 | d_loss:0.066 | g_loss:-0.509
iteration:2403 | d_loss:0.067 | g_loss:-0.509
iteration:2404 | d_loss:0.066 | g_loss:-0.509
iteration:2405 | d_loss:0.066 | g_

iteration:2568 | d_loss:0.056 | g_loss:-0.534
iteration:2569 | d_loss:0.057 | g_loss:-0.535
iteration:2570 | d_loss:0.057 | g_loss:-0.535
iteration:2571 | d_loss:0.058 | g_loss:-0.535
iteration:2572 | d_loss:0.058 | g_loss:-0.535
iteration:2573 | d_loss:0.057 | g_loss:-0.535
iteration:2574 | d_loss:0.057 | g_loss:-0.536
iteration:2575 | d_loss:0.055 | g_loss:-0.536
iteration:2576 | d_loss:0.055 | g_loss:-0.535
iteration:2577 | d_loss:0.058 | g_loss:-0.536
iteration:2578 | d_loss:0.056 | g_loss:-0.536
iteration:2579 | d_loss:0.056 | g_loss:-0.536
iteration:2580 | d_loss:0.057 | g_loss:-0.537
iteration:2581 | d_loss:0.056 | g_loss:-0.536
iteration:2582 | d_loss:0.056 | g_loss:-0.536
iteration:2583 | d_loss:0.057 | g_loss:-0.537
iteration:2584 | d_loss:0.056 | g_loss:-0.537
iteration:2585 | d_loss:0.055 | g_loss:-0.537
iteration:2586 | d_loss:0.056 | g_loss:-0.537
iteration:2587 | d_loss:0.056 | g_loss:-0.537
iteration:2588 | d_loss:0.055 | g_loss:-0.537
iteration:2589 | d_loss:0.055 | g_

iteration:2750 | d_loss:0.075 | g_loss:-0.577
iteration:2751 | d_loss:0.074 | g_loss:-0.577
iteration:2752 | d_loss:0.074 | g_loss:-0.577
iteration:2753 | d_loss:0.075 | g_loss:-0.578
iteration:2754 | d_loss:0.075 | g_loss:-0.578
iteration:2755 | d_loss:0.075 | g_loss:-0.578
iteration:2756 | d_loss:0.075 | g_loss:-0.579
iteration:2757 | d_loss:0.076 | g_loss:-0.579
iteration:2758 | d_loss:0.077 | g_loss:-0.580
iteration:2759 | d_loss:0.076 | g_loss:-0.580
iteration:2760 | d_loss:0.075 | g_loss:-0.580
iteration:2761 | d_loss:0.076 | g_loss:-0.581
iteration:2762 | d_loss:0.076 | g_loss:-0.581
iteration:2763 | d_loss:0.075 | g_loss:-0.581
iteration:2764 | d_loss:0.077 | g_loss:-0.582
iteration:2765 | d_loss:0.076 | g_loss:-0.582
iteration:2766 | d_loss:0.076 | g_loss:-0.582
iteration:2767 | d_loss:0.077 | g_loss:-0.582
iteration:2768 | d_loss:0.075 | g_loss:-0.582
iteration:2769 | d_loss:0.076 | g_loss:-0.583
iteration:2770 | d_loss:0.077 | g_loss:-0.583
iteration:2771 | d_loss:0.077 | g_

iteration:2934 | d_loss:0.071 | g_loss:-0.597
iteration:2935 | d_loss:0.071 | g_loss:-0.597
iteration:2936 | d_loss:0.070 | g_loss:-0.597
iteration:2937 | d_loss:0.071 | g_loss:-0.597
iteration:2938 | d_loss:0.070 | g_loss:-0.597
iteration:2939 | d_loss:0.071 | g_loss:-0.597
iteration:2940 | d_loss:0.070 | g_loss:-0.597
iteration:2941 | d_loss:0.070 | g_loss:-0.597
iteration:2942 | d_loss:0.070 | g_loss:-0.597
iteration:2943 | d_loss:0.070 | g_loss:-0.597
iteration:2944 | d_loss:0.070 | g_loss:-0.597
iteration:2945 | d_loss:0.070 | g_loss:-0.596
iteration:2946 | d_loss:0.070 | g_loss:-0.597
iteration:2947 | d_loss:0.070 | g_loss:-0.596
iteration:2948 | d_loss:0.070 | g_loss:-0.596
iteration:2949 | d_loss:0.070 | g_loss:-0.596
iteration:2950 | d_loss:0.070 | g_loss:-0.596
iteration:2951 | d_loss:0.070 | g_loss:-0.596
iteration:2952 | d_loss:0.070 | g_loss:-0.596
iteration:2953 | d_loss:0.070 | g_loss:-0.596
iteration:2954 | d_loss:0.070 | g_loss:-0.596
iteration:2955 | d_loss:0.070 | g_

### Statistical inference

#### Statistics of the data

In [15]:
mean_data = np.mean(processed_data, axis = 0)
variance_data = np.mean((processed_data - mean_data )* (processed_data- mean_data ), axis = 0)
skewness_data = np.mean((processed_data - mean_data )* (processed_data- mean_data ) * (processed_data- mean_data ), axis = 0)

In [16]:
print(mean_data)
print('------------')
print(variance_data)
print('------------')
print(skewness_data)

[[0.5        0.49547619 0.5014881 ]
 [0.50002976 0.50017857 0.        ]
 [0.         0.         0.        ]]
------------
[[0.02188512 0.00780811 0.00074183]
 [0.01493125 0.04641009 0.        ]
 [0.         0.         0.        ]]
------------
[[0.00249854 0.00381854 0.00036871]
 [0.00011419 0.00261314 0.        ]
 [0.         0.         0.        ]]


#### Statistics of the generated data

In [17]:
gaussian_noise= np.random.normal(0.5, 0.25, (batch_size, X_train.shape[1], X_train.shape[2], 1))
gen_data = np.squeeze(generator.predict(gaussian_noise))

In [18]:
mean_gen_data = np.mean(gen_data, axis = 0)
variance_gen_data = np.mean((processed_data - mean_gen_data)* (processed_data- mean_gen_data), axis = 0)
skewness_gen_data = np.mean((processed_data - mean_gen_data)* (processed_data- mean_gen_data) * (processed_data- mean_gen_data), axis = 0)

In [19]:
print(mean_gen_data)
print('------------')
print(variance_gen_data)
print('------------')
print(skewness_gen_data)

[[ 1.0428712  -0.85798156  0.91448504]
 [ 1.1955582   0.9390683  -0.37148285]
 [ 1.2076294  -0.13540411  0.15105346]]
------------
[[0.3165943  1.839656   0.17130831]
 [0.49869104 0.2390343  0.13799951]
 [1.45836887 0.01833427 0.02281715]]
------------
[[-1.93133006e-01  2.51485109e+00 -7.09938480e-02]
 [-3.67509827e-01 -1.43034387e-01  5.12644501e-02]
 [-1.76116918e+00  2.48253592e-03 -3.44660901e-03]]
